<a href="https://colab.research.google.com/github/dh610/Advanced-AI/blob/main/AirNet_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
#!pip install colab_ssh --upgrade

In [11]:
'''
from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password="1234")
'''

'\nfrom colab_ssh import launch_ssh_cloudflared, init_git_cloudflared\nlaunch_ssh_cloudflared(password="1234")\n'

In [6]:
import sys
import os

sys.path.append("/content/drive/MyDrive/Advanced-AI/Restormer/")
os.chdir("/content/drive/MyDrive/Advanced-AI/")
pwd = "/content/drive/MyDrive/Advanced-AI/AirNet/"
!ls

InstructIR		Practice1.ipynb  Restormer		test.ipynb
InstructIR_Colab.ipynb	README.md	 Restormer_train.ipynb


In [ ]:
!pip install lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 16.5 MB/s eta 0:00:00


In [ ]:
!source train.sh "Deraining/Options/Deraining_Restormer.yml"

/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py:208: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  main()
W0923 07:11:09.662000 134061473882752 torch/distributed/run.py:779] 
W0923 07:11:09.662000 134061473882752 torch/distributed/run.py:779] *****************************************
W0923 07:11:09.662000 134061473882752 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0923 07:11:09.662000 134061473882752 torch/distributed/run.py:779] ************

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import yaml
import random
import torch

from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch
import torch.optim as optim
import torch.nn as nn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_dir = pwd + "RainTrainL/"

denoise_input_dir = data_dir + "gaussian_sigma_5/"
derain_input_dir = data_dir + "rain/"
target_dir = data_dir + "target/"


In [ ]:
class ImagePairDataset(Dataset):
    def __init__(self, input_dir, target_dir, transform=None):
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.transform = transform
        self.image_filenames = [f for f in os.listdir(input_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        input_image_path = os.path.join(self.input_dir, self.image_filenames[idx])
        input_image = Image.open(input_image_path).convert("RGB")

        target_image_path = os.path.join(self.target_dir, self.image_filenames[idx])
        target_image = Image.open(target_image_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

denoise_dataset = ImagePairDataset(input_dir=denoise_input_dir, target_dir=target_dir, transform=transform)
derain_dataset = ImagePairDataset(input_dir=derain_input_dir, target_dir=target_dir, transform=transform)

dataset = denoise_dataset + derain_dataset

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)




In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from runpy import run_path
import os
from torch.optim.lr_scheduler import CosineAnnealingLR

load_arch = run_path(os.path.join('basicsr', 'models', 'archs', 'restormer_arch.py'))
Restormer = load_arch['Restormer']

parameters = {
    'inp_channels': 3,
    'out_channels': 3,
    'dim': 48,
    'num_blocks': [4, 6, 6, 8],
    'num_refinement_blocks': 4,
    'heads': [1, 2, 4, 8],
    'ffn_expansion_factor': 2.66,
    'bias': False,
    'LayerNorm_type': 'WithBias',
    'dual_pixel_task': False
}

model = Restormer(**parameters)
model = model.cuda()


criterion = nn.L1Loss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.999), weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-6)

ModuleNotFoundError: No module named 'einops'

In [ ]:
from tqdm import tqdm

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()


  0%|          | 0/90 [00:08<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 42.81 MiB is free. Process 33388 has 39.51 GiB memory in use. Of the allocated memory 38.93 GiB is allocated by PyTorch, and 91.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

model.eval()
val_loss = 0.0
with torch.no_grad():
    for data in test_loader:
        inputs = data['input'].cuda()
        targets = data['target'].cuda()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        val_loss += loss.item()
val_loss /= len(test_loader)
print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')